https://jira.x5.ru/browse/CVMUPG-1141

In [1]:
name = 'CVMUPG-1141_bigbon_pa'

In [2]:
import sys
import pandas as pd
import pyspark.sql.functions as F
sys.path.append('/home/jovyan/cvm_upgrade/')
sys.path.append('/home/jovyan/cvm_upgrade/upgrade/')
sys.path.append('/home/jovyan/glow-byte-filters-pyspark')
sys.path.append('/home/jovyan/x5_some_tasks/Osipov/')
from upgrade.post_analysis2.tools.groups_metrics_aggregation import GroupsMetricsAggregation
from upgrade.post_analysis2.tools.guests_metrics_preprocessing import GuestsMetricsPreprocessing
from upgrade.post_analysis2.tools.post_analysis_params import PostAnalysisParams
from upgrade.post_analysis2.tools.metric_names_mapper import map_col_names_to_russian
from upgrade.post_analysis2.tools.experiment_participants import ExperimentParticipants
from upgrade.post_analysis2.tools.experiment_params import ExperimentParams
import datetime
from datetime import timedelta
from spark import *
from utils_osipov import *

%load_ext autoreload
%autoreload 2

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 500

In [3]:
sc, spark = restart_spark(name, 100, executor_memory="3G", executor_cores=5, driver_memory="12G",
                          additional_params={"spark.sql.shuffle.partitions": "200"})
sc.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/18 18:48:35 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


### 1: X-28-42-Mareven-BigBon-sof

In [4]:
campaign_id = 'X-28-42-Mareven-BigBon-sof'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        80782: "sms_viber_tg",
        80782: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-01-25',
    'date_communication_to': '2022-01-31',

    'wave_ids': [
        81240 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-01-25',
    'date_to': '2022-01-31', #дата постпериода
    'control_camp_wave_id': 80782,
    'plu_list': list(map(str, [3003111,2082200,4025330,4059568,4063579,4025331,2082204,4025363,4115113,
                               3368169,3003110,2082206,4113988,3368168,3468665,3468666,4115114,4113990])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 2 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [5]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [6]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [7]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [8]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
0_80782_sms_viber_cg,59944,"-6,307.20",1.35,"80,794.50",1.37,"82,041.00","1,473,892.55","29,395.00",259.00,51375,...,"168,770.00",0.01,0.00,0.13,"7,667.00",0.02,0.00,0.07,"4,140.00",0.04
1_80782_sms_viber_cg,14986,"-19,237.44",0.00,0.00,0.00,0.00,"367,935.03","7,321.00",21.00,12811,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')

### 2: U-28-BigBon-postavschik

In [10]:
campaign_id = 'U-28-BigBon-postavschik'
output_fin_report = f'{campaign_id}_fin_report.csv'
output_main_report = f'{campaign_id}_main_report.csv'

experiment_params = ExperimentParams(**{
    # Camp wave id нужно записать в виде мапы из id в префиксы.
    # Префиксы ни на что не влияют, просто для более красивого названия групп
    # Заполняются camp_wave_id. Если для контроля и теста одинаковый, то пишем два раза
    'map_camp_wave_ids_to_suffixes': {
        80928: "sms_viber_tg",
        80928: "sms_viber_cg"
    },

    # communication period with guest (sending sms, push, etc.)
    'date_communication_from': '2022-01-25',
    'date_communication_to': '2022-01-31',

    'wave_ids': [
        81250 #если нет, то -1
    ],

    'communication_costs': {
        'sms': 1.38,
        'viber': 1.32,
        'email': 0.0083
    }
})

post_analysis_params = PostAnalysisParams(**{
    'spark': spark,
    'date_from': '2022-01-25',
    'date_to': '2022-01-31', #дата постпериода
    'control_camp_wave_id': 80928,
    'plu_list': list(map(str, [3003111,2082200,4025330,4059568,4063579,4025331,2082204,4025363,4115113,
                               3368169,3003110,2082206,4113988,3368168,3468665,3468666,4115114,4113990])),
    'threshold_item_condition': 'plu_list', #есть категория еще
    'count_for_plu_list': 2 #механика от 2-х штук
})


assert post_analysis_params.control_camp_wave_id in experiment_params.map_camp_wave_ids_to_suffixes.keys()

In [11]:
experiment_guests = ExperimentParticipants(spark, experiment_params)
experiment_guests_communications = experiment_guests.extract()

In [12]:
guest_metrics_preparation = GuestsMetricsPreprocessing(spark, experiment_guests_communications, post_analysis_params)
experiment_guests_with_metrics = guest_metrics_preparation.evaluate()

In [13]:
groups_metrics_aggregation = GroupsMetricsAggregation(spark, experiment_guests_with_metrics, post_analysis_params)
groups_metrics, map_to_effects_table = groups_metrics_aggregation.evaluate()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


In [14]:
fin_report = map_col_names_to_russian(groups_metrics)
fin_report

,"Размер выборки, чел.","Затраты на коммуникацию и на баллы (сумма delta с учетом redemption), руб.","Затраты на коммуникацию, руб./гость","Затраты на коммуникацию, руб.","Софинансирование, руб./гость","Софинансирование, руб.","Покупка предложения(чек), товаров (шт)","Покупка предложения(категория), товаров (шт)","Покупка предложения(набор plu), товаров (шт)","Гостей, купившие что-нибудь",...,"Эффект Ebitda софинансирование(чек) на группу, руб.",Эффект Ebitda софинансирование(чек) на группу / Ebitda софинансирование(чек) группы,Значимость Ebitda софинансирование(категория),"Эффект Ebitda софинансирование(категория) на гостя, руб.","Эффект Ebitda софинансирование(категория) на группу, руб.",Эффект Ebitda софинансирование(категория) на группу / Ebitda софинансирование(категория) группы,Значимость Ebitda софинансирование(набор plu),"Эффект Ebitda софинансирование(набор plu) на гостя, руб.","Эффект Ebitda софинансирование(набор plu) на группу, руб.",Эффект Ebitda софинансирование(набор plu) на группу / Ebitda софинансирование(набор plu) группы
group_name,,,,,,,,,,,,,,,,,,,,,
1_80928_sms_viber_cg,11675,"-3,387.52",0.00,0.00,0.00,0.00,"170,166.05",652.00,2.00,9155,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0_80928_sms_viber_cg,79953,"63,975.55",1.36,"108,571.92",1.43,"114,697.50","1,205,852.66","4,747.00",210.00,63031,...,"-656,606.00",-0.04,0.00,0.38,"30,225.00",0.25,0.00,0.40,"31,571.00",0.57


In [15]:
fin_report['meta_post_analysis'] = str(post_analysis_params.__dict__)
fin_report['meta_experiment_params'] = str(experiment_params.__dict__)
groups_metrics['meta_post_analysis'] = str(post_analysis_params.__dict__)
groups_metrics['meta_experiment_params'] = str(experiment_params.__dict__)

groups_metrics.T.to_csv(output_main_report, encoding='utf-8-sig', sep=';')
fin_report.T.to_csv(output_fin_report, encoding='utf-8-sig', sep=';')

22/02/19 03:37:48 ERROR cluster.YarnClientSchedulerBackend: YARN application has exited unexpectedly with state FAILED! Check the YARN application logs for more details.
22/02/19 03:37:48 ERROR cluster.YarnClientSchedulerBackend: Diagnostics message: Attempt recovered after RM restartDue to executor failures all available nodes are blacklisted
22/02/19 03:37:48 ERROR client.TransportClient: Failed to send RPC RPC 6536954866604737434 to /192.168.234.58:42066: java.nio.channels.ClosedChannelException
java.nio.channels.ClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(...)(Unknown Source)
22/02/19 03:37:48 ERROR cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Sending RequestExecutors(0,0,Map(),Set()) to AM was unsuccessful
java.io.IOException: Failed to send RPC RPC 6536954866604737434 to /192.168.234.58:42066: java.nio.channels.ClosedChannelException
	at org.apache.spark.network.client.TransportClient$RpcChannelListener.handleFailure(TransportClient.jav